<a href="https://colab.research.google.com/github/destello501/destello501.github.io/blob/main/CMSOpenDataZtoMuMu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
#panda nos permite manipular el archivo csv https://pandas.pydata.org
import pandas as pd
#numpy nos permite hacer operaciones usando vectores https://numpy.org
import numpy as np
#plotly nos permite crear graficas interactivas https://plotly.com/python/
import plotly.express as px

from scipy.optimize import curve_fit

import plotly.graph_objects as go


%matplotlib inline
#usamos la liga que está directamente en la página.
particles = pd.read_csv('https://opendata.cern.ch/record/5208/files/Zmumu.csv',delimiter=',')
particles.head()


,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,-0.4987,-1,0.0712,3.4221
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,-1.9912,1,0.0515,0.0000
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,1.8833,1,-0.0876,0.0000
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,2.9703,-1,-0.0492,0.0000
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,-0.0428,-1,0.0442,0.0000


In [60]:
fig_5=px.histogram(particles['pt1'],particles['pt1'])
fig_5.show()

In [61]:
fig_5=px.histogram(particles['pt2'],particles['pt2'])
fig_5.show()

In [62]:
particles['pt_max'] = np.maximum(particles['pt1'], particles['pt2'])
px.histogram(particles, x='pt_max', nbins=100, title='Distribución de pt máximo entre ambos muones')


In [63]:
# Calcular la relación de aislamiento relativa
particles['iso1_rel'] = particles['iso1'] / particles['pt1']
fig = px.histogram(particles, x='iso1_rel', nbins=50,
                   title='Aislamiento relativo iso1 / pt1',
                   labels={'iso1_rel': 'Aislación relativa'})
fig.update_layout(bargap=0.1)
fig.show()

In [64]:
# Paso 1: Asegurarse de tener la columna Q1Q2
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Paso 2: Filtrar solo los pares de muones con carga opuesta (candidatos a Z)
z_candidates = particles[particles['Q1Q2'] == -1].copy()

# Paso 3: Verificar si hay valores negativos
print("¿Hay iso1 < 0?", (z_candidates['iso1'] < 0).sum())

# Paso 4: Graficar histograma de iso1
fig = px.histogram(z_candidates, x='iso1', nbins=50,
                   title='Aislación iso1 en candidatos al bosón Z (Q1*Q2 = -1)',
                   labels={'iso1': 'Aislación (iso1)'})
fig.update_layout(bargap=0.1, xaxis=dict(range=[0, 0.3]))
fig.show()


¿Hay iso1 < 0? 0


In [65]:
px.scatter(particles, x='pt1', y='pt2',
           title='Momento transversal entre los dos muones')

In [66]:
filtered = particles[(particles['pt1'] < 100) & (particles['pt2'] < 100)]
px.scatter(filtered, x='pt1', y='pt2',
           title='pt1 vs pt2 (filtrado < 100 GeV)')
###LO QUE NO DEBO HACER

In [67]:
particles['pt_ratio'] = particles['pt1'] / particles['pt2']
px.histogram(particles, x='pt_ratio', nbins=50, title='pt1 / pt2 ratio')


In [68]:
px.scatter(particles, x='eta1', y='eta2',
           title='Pseudorapidez entre los dos muones')


In [69]:
####################################
######################################
######################################### Calcular la masa invariante
delta_eta = particles['eta1'] - particles['eta2']
delta_phi = particles['phi1'] - particles['phi2']

mass_squared = 2 * particles['pt1'] * particles['pt2'] * (np.cosh(delta_eta) - np.cos(delta_phi))
particles['invariant_mass'] = np.sqrt(mass_squared)

#Graficar
fig = px.histogram(particles, x='invariant_mass', nbins=60,
                   title='Masa invariante de dos muones',
                   labels={'invariant_mass': 'Masa invariante [GeV]'})
fig.update_layout(bargap=0.05)
fig.show()


In [70]:


# Paso 1: Extraer las masas invariantes directamente del DataFrame
masas = particles['invariant_mass'].values  # Asegúrate de que ya calculaste 'invariant_mass'

# Paso 2: Construcción de histograma
bin_w = 1.0  # Ancho del bin
lo_edge = np.floor(masas.min() / bin_w) * bin_w
hi_edge = np.ceil(masas.max() / bin_w) * bin_w
bins = np.arange(lo_edge, hi_edge + bin_w, bin_w)
counts, edges = np.histogram(masas, bins=bins)
centros = 0.5 * (edges[:-1] + edges[1:])  # Centros de los bins

# Paso 3: Definir la función de Breit-Wigner y el modelo completo
def breit_wigner(m, M, Gamma):
    num = M**2 * Gamma**2
    denom = (m**2 - M**2)**2 + M**2 * Gamma**2
    return num / denom

def modelo(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma) + a + b * m  # señal + fondo

# Paso 4: Ajuste en la región del bosón Z (ej. entre 80 y 100 GeV)
mask = (centros > 80) & (centros < 100)
x_fit = centros[mask]
y_fit = counts[mask]
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))  # Errores estadísticos (Poisson)

# Parámetros iniciales para el ajuste
p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]

# Ajustar el modelo
popt, pcov = curve_fit(modelo, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)
M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

# Imprimir resultados
print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV")
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV")

# Paso 5: Generar curvas para graficar el modelo ajustado
m_plot = np.linspace(lo_edge, hi_edge, 2000)
total_y = modelo(m_plot, *popt)
signal_y = N_sig * breit_wigner(m_plot, M_Z, Gamma_Z)

# Paso 6: Graficar con Plotly
hist = go.Histogram(
    x=masas,
    xbins=dict(start=lo_edge, end=hi_edge, size=bin_w),
    name="Datos",
    opacity=0.75,
    marker=dict(color="rgba(0,0,200,0.65)")
)

total_fit = go.Scatter(
    x=m_plot, y=total_y,
    mode="lines", name="Ajuste total",
    line=dict(color="orange"))

signal_Z = go.Scatter(
    x=m_plot, y=signal_y,
    mode="lines", name="Señal Z",
    line=dict(color="green", dash="dash"))

fig = go.Figure(data=[hist, total_fit, signal_Z])
fig.update_layout(
    title="Masa invariante: histograma y ajuste del bosón Z (sin filtro de carga)",
    xaxis_title="Masa invariante [GeV/c²]",
    yaxis_title=f"Eventos por bin (Δm = {bin_w} GeV)",
    bargap=0.05,
    template="plotly_white"
)
fig.show()


M_Z = 90.888 ± 0.032 GeV
Gamma_Z = 4.125 ± 0.105 GeV


In [71]:
#Combinacion de cargas
particles['Q1Q2'] = particles['Q1'] * particles['Q2']
print(particles['Q1Q2'].value_counts())

Q1Q2
-1    9664
 1     336
Name: count, dtype: int64


In [72]:
#FILTRO 1: Excluir muones con la misma carga
particles['Q1Q2'] = particles['Q1'] * particles['Q2']
z_candidates = particles[particles['Q1Q2'] == -1].copy()



In [73]:

# Calcular Q1Q2 si aún no lo hiciste
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Filtrar pares con carga opuesta
mask_carga = (particles['Q1Q2'] == -1)

# Filtro de aislamiento (ejemplo: iso < 0.15)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)

# Filtro de momento razonable (por ejemplo entre 20 y 70 GeV)
mask_pt = (particles['pt1'] > 20) & (particles['pt1'] < 70) & \
          (particles['pt2'] > 20) & (particles['pt2'] < 70)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt

# Crear nuevo DataFrame filtrado
filtered_particles = particles[mask_total].copy()

# Calcular la masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

# Verifica visualmente
fig = px.histogram(filtered_particles, x='invariant_mass', nbins=60,
                   title='Masa invariante (muones filtrados)',
                   labels={'invariant_mass': 'Masa invariante [GeV]'})
fig.update_layout(bargap=0.05)
fig.show()


In [74]:
# Paso 1: Calcular Q1Q2
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Filtros estándar
mask_carga = (particles['Q1Q2'] == -1)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)
mask_pt = (particles['pt1'] > 20) & (particles['pt1'] < 70) & \
          (particles['pt2'] > 20) & (particles['pt2'] < 70)
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.8) & (pt_ratio < 1.25)
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)

# Nuevo: filtro Δφ > 2.6 rad
delta_phi = np.abs(particles['phi1'] - particles['phi2'])
delta_phi = np.where(delta_phi > np.pi, 2 * np.pi - delta_phi, delta_phi)
mask_dphi = (delta_phi > 2.6)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Paso 2: Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

masas = filtered_particles['invariant_mass'].values

# Paso 3: Histograma
bin_w = 1.0
lo_edge = np.floor(masas.min() / bin_w) * bin_w
hi_edge = np.ceil(masas.max() / bin_w) * bin_w
bins = np.arange(lo_edge, hi_edge + bin_w, bin_w)
counts, edges = np.histogram(masas, bins=bins)
centros = 0.5 * (edges[:-1] + edges[1:])

# Paso 4: Ajuste Breit-Wigner
def breit_wigner(m, M, Gamma):
    num = M**2 * Gamma**2
    denom = (m**2 - M**2)**2 + M**2 * Gamma**2
    return num / denom

def modelo(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma) + a + b * m

mask_fit = (centros > 80) & (centros < 100)
x_fit = centros[mask_fit]
y_fit = counts[mask_fit]
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))

p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]
popt, pcov = curve_fit(modelo, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)

M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV")
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV")

# Paso 5: Gráfica
m_plot = np.linspace(lo_edge, hi_edge, 2000)
total_y = modelo(m_plot, *popt)
signal_y = N_sig * breit_wigner(m_plot, M_Z, Gamma_Z)

hist = go.Histogram(x=masas, xbins=dict(start=lo_edge, end=hi_edge, size=bin_w),
                    name="Datos", opacity=0.75,
                    marker=dict(color="rgba(0,0,200,0.65)"))

total_fit = go.Scatter(x=m_plot, y=total_y, mode="lines",
                       name="Ajuste total", line=dict(color="orange"))
signal = go.Scatter(x=m_plot, y=signal_y, mode="lines",
                    name="Señal Z", line=dict(color="green", dash="dash"))

fig = go.Figure(data=[hist, total_fit, signal])
fig.update_layout(
    title="Masa invariante filtrada + Δφ > 2.6 rad + Ajuste Breit-Wigner",
    xaxis_title="Masa invariante [GeV/c²]",
    yaxis_title=f"Eventos por bin (Δm = {bin_w} GeV)",
    bargap=0.05,
    template="plotly_white"
)
fig.show()

M_Z = 90.899 ± 0.057 GeV
Gamma_Z = 4.107 ± 0.185 GeV


In [75]:
# Calcular Q1Q2 si no está hecho
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Filtro 1: carga opuesta
mask_carga = (particles['Q1Q2'] == -1)

# Filtro 2: aislamiento (muones aislados)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)

# Filtro 3: pt mínimo (no se filtra por arriba, solo por abajo)
mask_pt = (particles['pt1'] > 25) & (particles['pt2'] > 25)

# Filtro 4: simetría de momento transversal
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.5) & (pt_ratio < 2.0)

# Filtro 5: pseudorapidez razonable
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)

# Filtro 6: ángulo azimutal (espalda con espalda)
delta_phi = np.abs(particles['phi1'] - particles['phi2'])
delta_phi = np.where(delta_phi > np.pi, 2 * np.pi - delta_phi, delta_phi)
mask_dphi = (delta_phi > 2.6)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

masas = filtered_particles['invariant_mass'].values

# Histograma
bin_w = 1.0
lo_edge = np.floor(masas.min() / bin_w) * bin_w
hi_edge = np.ceil(masas.max() / bin_w) * bin_w
bins = np.arange(lo_edge, hi_edge + bin_w, bin_w)
counts, edges = np.histogram(masas, bins=bins)
centros = 0.5 * (edges[:-1] + edges[1:])

# Ajuste de Breit-Wigner + fondo lineal
def breit_wigner(m, M, Gamma):
    num = M**2 * Gamma**2
    denom = (m**2 - M**2)**2 + M**2 * Gamma**2
    return num / denom

def modelo(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma) + a + b * m

mask_fit = (centros > 80) & (centros < 100)
x_fit = centros[mask_fit]
y_fit = counts[mask_fit]
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))

p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]
popt, pcov = curve_fit(modelo, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)

M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV")
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV")

# Graficar
m_plot = np.linspace(lo_edge, hi_edge, 2000)
total_y = modelo(m_plot, *popt)
signal_y = N_sig * breit_wigner(m_plot, M_Z, Gamma_Z)

hist = go.Histogram(x=masas, xbins=dict(start=lo_edge, end=hi_edge, size=bin_w),
                    name="Datos", opacity=0.75,
                    marker=dict(color="rgba(0,0,200,0.65)"))

total_fit = go.Scatter(x=m_plot, y=total_y, mode="lines",
                       name="Ajuste total", line=dict(color="orange"))
signal = go.Scatter(x=m_plot, y=signal_y, mode="lines",
                    name="Señal Z", line=dict(color="green", dash="dash"))

fig = go.Figure(data=[hist, total_fit, signal])
fig.update_layout(
    title="Masa invariante con todos los filtros + ajuste Breit-Wigner",
    xaxis_title="Masa invariante [GeV/c²]",
    yaxis_title=f"Eventos por bin (Δm = {bin_w} GeV)",
    bargap=0.05,
    template="plotly_white"
)
fig.show()

M_Z = 90.894 ± 0.051 GeV
Gamma_Z = 4.076 ± 0.165 GeV


In [76]:
# Paso 1: Aplica todos los filtros como antes
particles['Q1Q2'] = particles['Q1'] * particles['Q2']
mask_carga = (particles['Q1Q2'] == -1)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)
mask_pt = (particles['pt1'] > 25) & (particles['pt2'] > 25)
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.5) & (pt_ratio < 2.0)
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)
delta_phi_raw = np.abs(particles['phi1'] - particles['phi2'])
delta_phi_raw = np.where(delta_phi_raw > np.pi, 2 * np.pi - delta_phi_raw, delta_phi_raw)
mask_dphi = (delta_phi_raw > 2.6)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Paso 2: Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

# Paso 3: Contar eventos
total_filtrados = len(filtered_particles)
z_candidatos = ((filtered_particles['invariant_mass'] > 80) &
                (filtered_particles['invariant_mass'] < 100)).sum()

# Paso 4: Mostrar resultados
print(f"Total de pares de muones que cumplen todos los filtros: {total_filtrados}")
print(f"Número de candidatos al bosón Z (80 < M < 100 GeV): {z_candidatos}")
if total_filtrados > 0:
    eficiencia = (z_candidatos / total_filtrados) * 100
    print(f"Porcentaje de candidatos Z entre los eventos filtrados: {eficiencia:.2f}%")


Total de pares de muones que cumplen todos los filtros: 3329
Número de candidatos al bosón Z (80 < M < 100 GeV): 3024
Porcentaje de candidatos Z entre los eventos filtrados: 90.84%


In [77]:
# Calcular Q1Q2
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Filtros físicos (como antes)
mask_carga = (particles['Q1Q2'] == -1)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)
mask_pt = (particles['pt1'] > 25) & (particles['pt2'] > 25)
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.5) & (pt_ratio < 2.0)
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)
delta_phi_raw = np.abs(particles['phi1'] - particles['phi2'])
delta_phi_raw = np.where(delta_phi_raw > np.pi, 2 * np.pi - delta_phi_raw, delta_phi_raw)
mask_dphi = (delta_phi_raw > 2.6)

# Aplicar todos los filtros (excepto el de masa)
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

# Contar eventos
total_filtrados = len(filtered_particles)
z_candidatos = ((filtered_particles['invariant_mass'] > 80) &
                (filtered_particles['invariant_mass'] < 100)).sum()

# Mostrar resultados
print(f"Total de pares de muones que cumplen todos los filtros (sin filtrar por masa): {total_filtrados}")
print(f"De esos, los candidatos al bosón Z (80 < M < 100 GeV): {z_candidatos}")
if total_filtrados > 0:
    eficiencia = (z_candidatos / total_filtrados) * 100
    print(f"Porcentaje de candidatos Z entre eventos filtrados: {eficiencia:.2f}%")


Total de pares de muones que cumplen todos los filtros (sin filtrar por masa): 3329
De esos, los candidatos al bosón Z (80 < M < 100 GeV): 3024
Porcentaje de candidatos Z entre eventos filtrados: 90.84%


In [78]:


# Calcular Q1Q2
particles['Q1Q2'] = particles['Q1'] * particles['Q2']

# Filtros físicos
mask_carga = (particles['Q1Q2'] == -1)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)
mask_pt = (particles['pt1'] > 25) & (particles['pt2'] > 25)  # solo límite inferior
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.5) & (pt_ratio < 2.0)
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)

# Filtro angular Δφ > 2.6 rad
delta_phi_raw = np.abs(particles['phi1'] - particles['phi2'])
delta_phi_raw = np.where(delta_phi_raw > np.pi, 2 * np.pi - delta_phi_raw, delta_phi_raw)
mask_dphi = (delta_phi_raw > 2.6)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

# 🔥 NUEVO: limitar análisis a la región del bosón Z
filtered_particles = filtered_particles[(filtered_particles['invariant_mass'] > 89.5) &
                                        (filtered_particles['invariant_mass'] < 93.5)].copy()

masas = filtered_particles['invariant_mass'].values

# Histograma
bin_w = 1.0
lo_edge = 80
hi_edge = 100
bins = np.arange(lo_edge, hi_edge + bin_w, bin_w)
counts, edges = np.histogram(masas, bins=bins)
centros = 0.5 * (edges[:-1] + edges[1:])

# Breit-Wigner + fondo lineal
def breit_wigner(m, M, Gamma):
    num = M**2 * Gamma**2
    denom = (m**2 - M**2)**2 + M**2 * Gamma**2
    return num / denom

def modelo(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma) + a + b * m

# Ajuste sobre la región [80, 100]
x_fit = centros
y_fit = counts
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))

p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]
popt, pcov = curve_fit(modelo, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)

M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV")
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV")

# Gráfica
m_plot = np.linspace(lo_edge, hi_edge, 2000)
total_y = modelo(m_plot, *popt)
signal_y = N_sig * breit_wigner(m_plot, M_Z, Gamma_Z)

hist = go.Histogram(x=masas, xbins=dict(start=lo_edge, end=hi_edge, size=bin_w),
                    name="Datos", opacity=0.75,
                    marker=dict(color="rgba(0,0,200,0.65)"))

total_fit = go.Scatter(x=m_plot, y=total_y, mode="lines",
                       name="Ajuste total", line=dict(color="orange"))
signal = go.Scatter(x=m_plot, y=signal_y, mode="lines",
                    name="Señal Z", line=dict(color="green", dash="dash"))

fig = go.Figure(data=[hist, total_fit, signal])
fig.update_layout(
    title="Masa invariante filtrada en [80, 100] GeV + ajuste Breit-Wigner",
    xaxis_title="Masa invariante [GeV/c²]",
    yaxis_title=f"Eventos por bin (Δm = {bin_w} GeV)",
    bargap=0.05,
    template="plotly_white"
)
fig.show()


M_Z = 91.025 ± 0.010 GeV
Gamma_Z = 0.002 ± 17.642 GeV


In [79]:
# Paso 1: Aplica todos los filtros como antes
particles['Q1Q2'] = particles['Q1'] * particles['Q2']
mask_carga = (particles['Q1Q2'] == -1)
mask_iso = (particles['iso1'] < 0.15) & (particles['iso2'] < 0.15)
mask_pt = (particles['pt1'] > 25) & (particles['pt2'] > 25)
pt_ratio = particles['pt1'] / particles['pt2']
mask_balance = (pt_ratio > 0.5) & (pt_ratio < 2.0)
mask_eta = (abs(particles['eta1']) < 2.4) & (abs(particles['eta2']) < 2.4)
delta_phi_raw = np.abs(particles['phi1'] - particles['phi2'])
delta_phi_raw = np.where(delta_phi_raw > np.pi, 2 * np.pi - delta_phi_raw, delta_phi_raw)
mask_dphi = (delta_phi_raw > 2.6)

# Combinar todos los filtros
mask_total = mask_carga & mask_iso & mask_pt & mask_balance & mask_eta & mask_dphi
filtered_particles = particles[mask_total].copy()

# Paso 2: Calcular masa invariante
delta_eta = filtered_particles['eta1'] - filtered_particles['eta2']
delta_phi = filtered_particles['phi1'] - filtered_particles['phi2']
mass_squared = 2 * filtered_particles['pt1'] * filtered_particles['pt2'] * \
               (np.cosh(delta_eta) - np.cos(delta_phi))
filtered_particles['invariant_mass'] = np.sqrt(mass_squared)

# Paso 3: Contar eventos
total_filtrados = len(filtered_particles)
z_candidatos = ((filtered_particles['invariant_mass'] > 80) &
                (filtered_particles['invariant_mass'] < 100)).sum()

# Paso 4: Mostrar resultados
print(f"Total de pares de muones que cumplen todos los filtros: {total_filtrados}")
print(f"Número de candidatos al bosón Z (80 < M < 100 GeV): {z_candidatos}")
if total_filtrados > 0:
    eficiencia = (z_candidatos / total_filtrados) * 100
    print(f"Porcentaje de candidatos Z entre los eventos filtrados: {eficiencia:.2f}%")


Total de pares de muones que cumplen todos los filtros: 3329
Número de candidatos al bosón Z (80 < M < 100 GeV): 3024
Porcentaje de candidatos Z entre los eventos filtrados: 90.84%
